## Eluwa training notebook

This is a straightforward mash-up of two sources - [a tutorial notebook on retraining OPT](https://colab.research.google.com/drive/1jCkpikz0J2o20FBQmYmAGdiKmJGOMo-o) and settings + data from the [Stanford Alpaca](https://github.com/tatsu-lab/stanford_alpaca) github.

### Install requirements

First, run the cells below to install the requirements:

In [ ]:
!pip install -q bitsandbytes datasets accelerate loralib transformers peft
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM


### Model loading

Here let's load the `opt` model and tokenizer from their huggingface link. The model is loaded in [8bit](https://) mode. This drastically reduces the amount of memory required to run the model. Without it, any attempt to train models above a certain size (say, something like `pythia1b`) will max out the available RAM/VRAM and get your nowhere. To understand what's going on with that `8bit` flag, it might be useful to read [this first](https://https://huggingface.co/blog/hf-bitsandbytes-integration).

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "facebook/opt-1.3b", 
    load_in_8bit=True, 
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("facebook/opt-1.3b")

### Training
Here we load the alpaca dataset. 

In [ ]:
import transformers
from datasets import load_dataset
data = load_dataset("tatsu-lab/alpaca")
data = data.map(lambda samples: tokenizer(samples['instruction']), batched=True)


### Post-processing on the model

We need to apply some post-processing on the 8-bit model to enable training, let's freeze all our layers, and cast the layer-norm in `float32` for stability. We also cast the output of the last layer in `float32` for the same reasons.

In [ ]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Apply LoRA

A LoRA (Low-Rank Adapter) is a way of training a portion of a model, instead of training the entire model. In inference, the LoRA then fits 'on top' of the existing model to modify its outputs.  It takes only a fraction of the memory required to store. I highly recommend [reading the paper](https://arxiv.org/pdf/2106.09685.pdf): without it we'd be stuck retraining entire models from scratch every time. 

This magic happens with `peft`! Let's load a `PeftModel` and specify that we are going to use low-rank adapters (LoRA) using `get_peft_model` utility function from `peft`. This code will output how many parameters we can actually train here with this method. 

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

from peft import LoraConfig, get_peft_model 

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)


### Defining the training arguments
The `trainer` function here specifies important things. [The documentation](https://huggingface.co/transformers/v3.0.2/main_classes/trainer.html#transformers.TrainingArguments) covers all these parameters and then some. 
Things you typically want to pay attention to:
 is the 

*   `learning rate` (3e-4 is commonly seen, although that may have been a joke by Karpathy) 
*   `num_train_epochs`: a measure of how long you want to train your model for. Each time a dataset passes through an algorithm, it is said to have completed an epoch. `max_steps ` is an alternate way of controlling how long you're going to train for. Helps to comment out one if you're using the other.



In [ ]:

trainer = transformers.Trainer(
    model=model, 
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=8, 
        gradient_accumulation_steps=4,
        warmup_steps=100, 
        num_train_epochs=1,
        #max_steps=1000, 
        learning_rate=2e-4, 
        fp16=True,
        logging_steps=10, 
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!

###The actual training

I keep `resume_from_checkpoint = False` because the function can't seem to handle quantized models very well. This does mean that you're starting from scratch every time. Colab has an annoying habit of restarting runtimes out of nowhere, so if you're training for a long time, say a prayer before you press the button.

In [ ]:
trainer.train(resume_from_checkpoint = False)

### Saving your LoRA

Important! Remember to run this so you can save and download your LoRa. The training process will generate .bin files, but they aren't the models you're looking for.



In [ ]:
model.save_pretrained("lora-eluwa-opt")